# 装饰器

- 装饰模式有很多经典的使用场景，例如插入日志、性能测试、事务处理等等，有了装饰器，就可以提取大量函数中与本身功能无关的类似代码，从而达到代码重用的目的。

- 装饰器本身是一个Python函数，他可以让其他函数在不需要做任何代码变动的前提下增加额外功能，装饰器的返回值也是一个额外的（函数）对象。

## 为什么要使用装饰器

- 一般在开发过程中，要遵循开放封闭原则，虽然在这个原则是用的面向对象开发，但是也适用于函数式编程，简单来说，它规定已经实现的功能代码不允许被修改，但可以被扩展，这时就要使用装饰器了。

## 使用装饰器来修饰函数的好处

- 不改变原有的程序，并且可以添加新的功能
- 可以提高程序的可重复利用性，并增加了程序的可读性。

## 装饰器函数特点：

1. 函数里面嵌套函数，外层函数接受一个参数(被装饰的函数)
2. 外层函数返回的是内层嵌套函数, 内层中调用了外部接收到的函数(即传入的参数，也就是被装饰的函数)

In [4]:
# 使用闭包实现装饰器功能

def test(func):
    def wrapper():
        print("----这是新添加的功能----")
        func()
    return wrapper


def foo1():
    print('----这是foo1函数----')
    
# 未被装饰前的函数功能
foo1()
print("*"*20)


# 被装饰后
foo1 = test(foo1)
foo1()


----这是foo1函数----
********************
----这是新添加的功能----
----这是foo1函数----


In [9]:
# 使用装饰器语法糖 @

def test(func):
    def wrapper():
        print("----这是新添加的功能----")
        func()
    return wrapper

@test
def foo1():
    print('----这是foo1函数----')
    

foo1()

----这是新添加的功能----
----这是foo1函数----


## 装饰器何时执行

- **装饰器是在函数被装饰时，就已经调用，而不是被装饰函数调用时，对应的装饰器才被调用**


- 如下所示，被装饰函数 foo1 未调用，装饰器内容就被打印

In [12]:
def test(func):
    print("----这是一个装饰器----")
    def wrapper():
        print('----这是新的功能----')
        func()
    return wrapper

@test
def foo1():
    print('----这是foo1函数----')


----这是一个装饰器----


## 多个装饰器装饰同一个函数时的运行顺序

- 当被装饰函数执行时，装饰器的执行顺序是从里到外，最先调用最里层的装饰器，最后调用最外层的装饰器，
- 等效于`foo = test1(test2(foo))`

In [27]:
def test1(func):
    def wrapper():
        return "功能1" + func()
    return wrapper

def test2(func):
    def wrapper():
        return "功能2" + func()
    return wrapper

@test1
@test2
def foo():
    return "foo"

print(foo())

功能1功能2foo


## 装饰器装饰带有参数的函数

- 上面的案例被装饰的函数都是没有参数的，如何装饰带有参数的函数，如下

In [29]:
# 被装饰函数带有参数
def test(func):
    def wrapper(a, b, c):
        print("---这是新功能---")
        func(a, b, c)
    return wrapper

@test
def foo(a, b, c):
    print(a, b, c)

foo(1, 2, 3)

---这是新功能---
1 2 3


In [34]:
# 对上面的代码进行改进
# 因为不知道具体需要装饰的函数有什么参数，所以可以使用 （*args, **kwargs) 来接收参数
# 通用装饰器
def test(func):
    def wrapper(*args, **kwargs):
        print('---这是新功能---')
        ret = func(*args, **kwargs)
        return ret
    return wrapper


@test
def foo1(a, b, c, d=None):
    print(a, b, c, d)

@test
def foo2(a, b):
    print(a, b)
    
    
foo1(1, 2, 3, 4)
foo2(10, 20)

---这是新功能---
1 2 3 4
---这是新功能---
10 20


- **装饰器本质，其实是改变了被装饰函数的原有引用，使得原本指向 `foo` 函数的引用， 指向了 `wrapper` **
- 调用 `foo` 实际上就是调用 `wrapper` ，所以传参传给 `wrapper` 就行

## 带参数的装饰器

- 在原有装饰器的基础上，设置外部变量
- 在原有装饰器外包裹一层函数，该函数返回装饰器的引用
- 这样可以通过传入不同的参数，让装饰器执行不同的功能。

In [38]:
import datetime

def func_decorator(arg):
    def test(func):
        def wrapper(*args, **kwargs):
            if arg == 'foo1':
                print("这是给函数foo1添加的功能")
                print("当前时间:", datetime.datetime.now())
                ret = func(*args, **kwargs)
            else:
                print("这是给函数{}添加的功能".format(arg))
                ret = func(*args, **kwargs)
                
            return ret
        return wrapper
    return test


@func_decorator('foo1')
def foo1():
    print("这是函数foo1")
    

@func_decorator('foo2')
def foo2():
    print("这是函数foo2")
    
foo1()
print('*'*20)
foo2()

这是给函数foo1添加的功能
当前时间: 2019-06-02 22:25:35.647567
这是函数foo1
********************
这是给函数foo2添加的功能
这是函数foo2


- 带有参数的装饰器执行过程：
    1. 先执行函数 `func_decorator('foo1')` 返回一个指向装饰器`test`的引用 test
    2. @test 装饰器装饰函数`foo1`

## 装饰器(decorator)功能

1. 引⼊⽇志
2. 函数执⾏时间统计
3. 执⾏函数前预备处理
4. 执⾏函数后清理功能
5. 权限校验等场景
6. 缓存

## Python内置装饰器

- 在Python中有三个内置的装饰器，都是跟class相关的：staticmethod、classmethod 和property。
    - staticmethod 是类静态方法，其跟成员方法的区别是没有 self 参数，并且可以在类不进行实例化的情况下调用
    - classmethod 与成员方法的区别在于所接收的第一个参数不是 self （类实例的指针），而是cls（当前类的具体类型）
    - property 是属性的意思，表示可以通过通过类实例直接访问的信息

## 类装饰器

- 装饰器函数其实是这样⼀个接⼝约束，它必须接受⼀个callable对象作为参数，然后返回⼀个callable对象。在Python中⼀般callable对象都是函数，但也有例外。只要某个对象重写了 `__call__()` ⽅法，那么这个对象就是callable的。

- 关于 `__call__` 方法，不得不先提到一个概念，就是可调用对象（callable），我们平时自定义的函数、内置函数和类都属于可调用对象，但凡是可以把一对括号()应用到某个对象身上都可称之为可调用对象，判断对象是否为可调用对象可以用函数 callable
- 如果在类中实现了 `__call__` 方法，那么实例对象也将成为一个可调用对象

In [3]:
class Test(object):
    def __call__(self):
        print(self)
        print("call me")
        return "__call__"

        
t = Test()

t()

call me


'__call__'

In [7]:
class Test(object):
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print("这是新添加的功能")
        ret = self.func(*args, **kwargs)
        
        return ret
    

@Test
def foo(a, b):
    return a*b

foo(10, 20)

这是新添加的功能


200

- @Test可以理解为 `foo = Test(foo)` 返回一个 Test 类的对象 foo， 执行这个对象时会调用 `__call__` 方法